# 7. Búsqueda en noticias en Sophia2


Este notebook muestra una manera de extraer noticias desde la base de datos Sophia2, a partir de una **palabra clave**, un **país** y un **periodo de tiempo**.

_Disclaimer_: La base de datos Sophia2 se encuentra en fase de mejoramiento continuo. Por cualquier pregunta o problema técnico con este notebook, no dudar en contactarme mvernier@inf.uach.cl

## 1. Definir los parámetros

In [1]:
## SE PUEDE MODIFICAR LOS VALORES

keyword="biodiversidad"
country="colombia"
from_="2018-01-01" ##YYYY-MM-DD
to_="2019-01-01" ##YYYY-MM-DD

## 2. Realizar la búsqueda

In [2]:
#NO MODIFICAR
from datetime import datetime
from elasticsearch import Elasticsearch

IP = "45.56.113.162"
PORT = 9200

es = Elasticsearch(
    IP,
    port=PORT,
)

query = {
  "query": { 
    "bool": { 
      "must": [
        {"match": { "text":keyword}}
        
      ],
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }}},
        { "term":  { "country": country }} 
    ]
    }
  }
}

In [3]:
## NO SE PUEDE DEVOLVER MAS DE 10.000 noticias. 
##En caso de obtener 10.000 noticias, se recomienda buscar en un periodo de tiempo más corto.

res = es.search(index="news", body=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

import pandas as pd

data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[],'search':[]}

df = pd.DataFrame(data)  
  
for hit in res['hits']['hits']:
    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    search = keyword
    
    new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date, 'search':search}
    
    df = df.append(new_row, ignore_index=True)


Son 0 noticias encontradas...


## 3. Verificar si existen noticias duplicadas y eliminarlas

- ¿Existen noticias duplicadas (con la misma URL) en los resultados?

In [51]:
df['url'].duplicated().any()

True

- ¿Cuáles son las noticias duplicadas?

In [52]:
duplicated=df[df['url'].duplicated()]
print("Son "+str(len(duplicated))+" noticias duplicadas.")
duplicated

Son 442 noticias duplicadas.


,id_news,country,media_outlet,url,title,text,date,search
5,5306786.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/pre...,Premio Nobel indio dice que el cambio climátic...,"El científico indio Rajendra Pachauri, Nobel ...",2017-05-29,cambio climático
10,5294966.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/bea...,Beatriz Sánchez advierte que Chile es “muy vul...,En el foro “El gran reto de Chile ante el camb...,2017-09-07,cambio climático
11,5295324.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/bea...,Beatriz Sánchez advierte que Chile es “muy vul...,En el foro “El gran reto de Chile ante el camb...,2017-09-07,cambio climático
24,5295556.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/est...,Estos son los países con mayores y menores pro...,"Hace dos años, la Universidad de Notre Dame ...",2017-09-07,cambio climático
31,5305579.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/gob...,Gobierno chileno manifiesta su “profunda decep...,Una «profunda decepción» manifestó el gobiern...,2017-06-01,cambio climático
...,...,...,...,...,...,...,...,...
6893,6000473.0,chile,elciudadano,https://www.elciudadano.com/justicia/el-genero...,El género de cada uno… ¿y qué?,A fines de abril el Ministerio de Educación ...,2017-07-08,cambio climático
6898,5314695.0,chile,elciudadano,https://www.elciudadano.com/medio-ambiente/pro...,Provincia Heilongjiang prohibió transgénicos: ...,Es tradición que en los países que celebran e...,2017-02-04,cambio climático
6904,4233462.0,chile,elciudadano,https://www.elciudadano.com/politica/la-difere...,La Diferencia entre MEO y Alejandro Navarro,Durante años los ciudadanos y ciudadanas de C...,2017-10-18,cambio climático
6916,6023000.0,chile,elciudadano,https://www.elciudadano.com/justicia/el-haitia...,El haitiano que planteó la igualdad de las raz...,Antenor Firmin publicó en 1885 su tratado De ...,2017-01-22,cambio climático


- Eliminar las noticias duplicadas

In [53]:
df = df.drop_duplicates(subset='url', keep='first')

- ¿Cuántas noticias quedan?

In [54]:
print("Quedan "+str(len(df))+" noticias...")

Quedan 6479 noticias...


## 4. Guardar las noticias en un CSV

In [55]:
nombre_archivo=keyword+"_"+country+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(nombre_archivo)
df.to_csv("./"+nombre_archivo)

El archivo se llama:
cambio climático_chile_2017-01-01_2018-01-01.csv


## 5. Agrupar varios archivos CSV en un solo archivo 

In [40]:
archivos_entrada = [
    "biodiversidad_mexico_2010-01-01_2021-01-01.csv",
    "biodiversidad_colombia_2010-01-01_2021-01-01.csv"
]

archivo_salida="biodiversidad_mexico_colombia.csv"


In [41]:
## NO MODIFICAR
df_list=[]

for file in archivos_entrada:
    df_file=pd.read_csv(file)
    df_list.append(df_file)
    
df_final=pd.concat(df_list)
df_final.drop_duplicates(subset='url', keep='first')
df_final.to_csv(archivo_salida)

print("Se agrupó los archivos en el archivo "+archivo_salida+" y se suprimió las noticias duplicadas.")

Se agrupó los archivos en el archivo biodiversidad_mexico_colombia.csv y se suprimó las noticias duplicadas.
